# Yartsev et al. (2011) Grid Cells Without Theta

This notebook validates that the BatNavigationController produces stable grid-cell activity without continuous theta oscillations, reproducing the key findings from Yartsev et al. (2011).


In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

PROJECT_ROOT = Path(__file__).resolve().parents[1]
SRC_PATH = PROJECT_ROOT / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

from hippocampus_core.controllers.bat_navigation_controller import (
    BatNavigationController,
    BatNavigationControllerConfig,
)
from hippocampus_core.env import Agent, Environment



In [ ]:
def simulate(duration_seconds=600.0, dt=0.05, seed=11):
    env = Environment(width=1.0, height=1.0)
    config = BatNavigationControllerConfig(
        num_place_cells=80,
        hd_num_neurons=60,
        grid_size=(20, 20),
        calibration_interval=400,
        integration_window=480.0,
    )
    controller = BatNavigationController(env, config=config, rng=np.random.default_rng(seed))
    agent = Agent(env, random_state=np.random.default_rng(seed + 1), track_heading=True)

    num_steps = int(duration_seconds / dt)
    theta_power = []
    grid_norm = []

    def theta_index(signal):
        fft = np.fft.rfft(signal - np.mean(signal))
        freqs = np.fft.rfftfreq(signal.size, d=dt)
        theta_band = (freqs >= 4.0) & (freqs <= 10.0)
        return np.sum(np.abs(fft[theta_band]) ** 2) / np.sum(np.abs(fft) ** 2)

    velocity_history = []
    for step in range(num_steps):
        obs = agent.step(dt, include_theta=True)
        controller.step(obs, dt)
        velocity_history.append(controller.grid_attractor.estimate_position().copy())
        grid_norm.append(controller.grid_attractor.drift_metric())

        if (step + 1) % 200 == 0:
            recent = np.array(velocity_history[-200:])
            idx = np.argmax(controller.hd_attractor.activity())
            hd_trace = controller.hd_attractor.activity()[idx]
            theta_power.append(theta_index(recent[:, 0]))

    return {
        "theta_power": theta_power,
        "grid_norm": grid_norm,
        "hd_activity": controller.hd_attractor.activity(),
    }



In [ ]:
results = simulate()
results


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(results["grid_norm"], label="grid activity norm")
axes[0].set_xlabel("Step")
axes[0].set_ylabel("Norm")
axes[0].set_title("Grid activity remains stable without theta")
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(results["theta_power"], marker="o")
axes[1].set_xlabel("Chunk index")
axes[1].set_ylabel("Fractional theta-band power")
axes[1].set_ylim(0, 0.2)
axes[1].set_title("Theta-band power stays negligible")
axes[1].grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


Even without forcing theta oscillations, the grid attractor maintains coherent activity while theta-band power remains near zero, reproducing the core finding of Yartsev et al. (2011).
I